---
title: "Political Blogs Network Analysis with `graph-tool`"
description: "Tutorial on computational network analysis for Introduction to Computational Social Science with Python in the GESIS 2024 Fall Seminar in Computational Social Science"
author: "John McLevey"
date: "9/13/2024" # last modified
categories:
---


In this notebook, we'll recreate the political blogs network visualization from Adamic and Glance (2005), as well as a series of visualizations discussed in the lecture. You'll learn how to use graph-tool to fit nested Stochastic Block Models (SBMs) and create informative network visualizations.

This notebook assumes familiarity with the content covered in the recorded lecture. Be sure to watch that before working on this notebook!

> [!tip] Environment Setup
> For performance reasons, most of `graph-tool` is actually written in C++ and then wrapped in Python. This means that `graph-tool` has some non-Python dependencies that can't be installed using Python-only package managers such as pip or Poetry (which uses pip). The easiest way to install `graph-tool` is to create a conda virtual environment using the conda-forge channel. You can do this with:
>
> ```zsh
> conda create --name gt -c conda-forge graph-tool
> cd tutorials/4-computational-network-analysis/
> conda activate gt
> ```

We'll start by importing the necessary libraries, including `graph-tool`, and setting up the environment for inline visualizations.


In [ ]:
import numpy as np
import graph_tool.all as gt
import matplotlib as mpl
from icsspy.networks import rotate_positions

print(f'Using graph-tool version {gt.__version__}')

If you are running this code in a Quarto or Jupyter Notebook, you'll want to tell `matplotlib` to plot inline.

```python
%matplotlib inline
```

## Loading the Political Blogs Dataset

We can access the political blogs data directly from the graph-tool data repository.


In [ ]:
g = gt.collection.data["polblogs"]
print(g)

The graph object `g` includes metadata about the network, stored in an internal **property map**. We'll explore property maps in more detail later. For now, let's print the graph metadata.


In [ ]:
print(g.gp.readme)

## Reproduce the "Divided They Blog" Figure

![](../../slides/media/divided_they_blog.png)

To recreate the "Divided They Blog" figure, we need to:

- Access information about the binary political classification for each blog node.
- Assign different colors to each class using graph-tool.
- Plot the network using a force-directed layout algorithm.

According to the metadata, the political classification is stored in an internal property map called `value`, where `0` indicates `left-liberal` (Democrats) and `1` indicates `right-conservative` (Republicans). This information is stored in an internal vertex property map, accessible via `g.vp.value`. You can confirm its type with the following:


In [ ]:
type(g.vp.value)

We can easily look up the political class for any given node by passing its integer ID. For example, vertex 30:


In [ ]:
g.vp.value[30]

To view all classifications, we can iterate over the vertices and print each vertex ID followed by its class label:


In [ ]:
for v in g.vertices():
    print(v, g.vp.value[v])

### Assigning Colors Based on Political Classification

We need to create a mapping between colors and labels. To match the original figure, we'll use red (`#D72F32`) for right-conservative (Republicans) and blue (`#2F357E`) for left-liberal (Democrats).


In [ ]:
political_colors = {0: "#2F357E", 1: "#D72F32"}  # color map
vertex_political_colors = g.new_vertex_property("string")  # new vertex property

# assign colors to each vertex based on the political classification
for v in g.vertices():
    vertex_political_colors[v] = political_colors[g.vp.value[v]]

### Force-Directed Layout and Initial Visualization

To reproduce the layout in the figure, we'll use a force-directed layout with the default parameters. Let's see what we have:


In [ ]:
pos = gt.sfdp_layout(g)

gt.graph_draw(
    g, pos,
    vertex_fill_color=vertex_political_colors,
    output_size=(1200, 1200),
    bg_color=[1, 1, 1, 1], # white background (transparent by default)
    inline=True,
    # output="output/polblogs_1.png",
)

As you can see, there are many isolates in this figure. To create a cleaner and more informative visualization, we'll focus only on the giant component.


In [ ]:
giant = gt.extract_largest_component(g, directed=True)
giant

Recompute the positions for the giant component and visualize it:


In [ ]:
pos = gt.sfdp_layout(giant)

gt.graph_draw(
    giant, pos,
    vertex_fill_color=vertex_political_colors,
    output_size=(1200, 1200),
    bg_color=[1, 1, 1, 1],
    inline=True,
    # output="output/polblogs_1.png",
)

### Rotating the Layout for Better Visualization

To re-create the original figure (and improve aesthetics), we need to rotate the figure so that the two clusters are horizontally aligned. We can use the `rotate_positions()` function from the course package to do this:


In [ ]:
pos = rotate_positions(pos, a=90)

gt.graph_draw(
    giant, pos,
    vertex_fill_color=vertex_political_colors,
    output_size=(1200, 1200),
    bg_color=[1, 1, 1, 1],
    inline=True,
    # output="output/polblogs_2.png",
)

### Fitting the First Nested SBM

To recreate the next figure, we need to fit our first Nested SBM using the `minimize_nested_blockmodel_dl()` function. This function selects the block partition with the lowest description length. The result is saved to blockstate:


In [ ]:
blockstate = gt.minimize_nested_blockmodel_dl(giant)

We can retrieve information about the block assignments at different levels of the nested SBM by specifying the level. Here, we'll use the lowest level, `level 0`:


In [ ]:
blockstate_level_0 = blockstate.levels[0]
blockstate_level_0

### Visualizing the Blockstate

`graph-tool` provides a convenient drawing method for blockstates. We can visualize the block assignments as follows:


In [ ]:
blockstate_level_0.draw(
    pos=pos,
    output_size=(1200, 1200),
    bg_color=[1, 1, 1, 1],
    # output="../output/polblogs_3.png",
    inline=True
)

While the results may vary slightly due to the stochastic nature of the model, we have effectively recreated the figure!

### Refining the Layout Based on SBM Grouping

As a refinement step **based on model criticism**, we'll adjust the force-directed layout by adding an attractive force between nodes in the same block. This is done by passing the following arguments to `sfdp_layout()`:

- `groups`: A vertex property map that assigns nodes to specific groups, in this case, block assignments at the lowest level of the nested SBM (`blockstate_level_0.b`). This adds additional attractive forces for block membership in the layout.
- `gamma`: Controls the strength of the attractive force for nodes in the same block. A small value corresponds to a weak force and more spread out clusters, while a larger value results in more compact clusters.

We'll pass `blockstate_level_0.b` (block assignments) to `sfdp_layout()`. The `.b` attribute provides access to the block assignments.


In [ ]:
pos_refined = gt.sfdp_layout(g, groups=blockstate_level_0.b, gamma=.04)

We'll use `pos_refined` for our next `draw()` and customize the edge colors for better readability:

- `edge_gradient=[]` disables edge gradients, using a solid color instead.
- `edge_color="#33333322"`: Sets the edge color to a semi-transparent gray.


In [ ]:
blockstate_level_0.draw(
    pos=pos_refined,
    edge_gradient=[],
    edge_color="#33333322",
    bg_color=[1, 1, 1, 1],
    output_size=(1200, 1200),
    inline=True,
    # output="../output/polblogs_4.png",
)

### Rotating the Final Visualization

Let's rotate the refined layout for a more aesthetically pleasing figure:


In [ ]:
pos_refined = rotate_positions(pos_refined, 90)

blockstate_level_0.draw(
    pos=pos_refined,
    edge_gradient=[],
    edge_color="#33333322",
    bg_color=[1, 1, 1, 1],
    output_size=(1200, 1200),
    inline=True,
    # output="../output/polblogs_4.png",
)

## Chordal Diagram with Hierarchical Edge Bundling

To wrap up this first (of three) `graph-tool` tutorials, we'll develop a couple of chordal figure with hierarchical edge bundling. This is straightforward using the blockstate .draw() method, which defaults to this type of visualization:


In [ ]:
blockstate.draw(
    output_size=(1200, 1200),
    bg_color=[1, 1, 1, 1],
    # output="../output/polblogs_5.png"
)

### Adjusting Node Colors and Exploring Hierarchy

For the final adjustment, let's assign node colors based on political classification rather than block membership. The block structure will still be evident from the layout, but this will help us see the extent to which the binary classification lines up with the hierarchical blockmodel.


In [ ]:
blockstate.draw(
    vertex_fill_color=vertex_political_colors,
    output_size=(1200, 1200),
    bg_color=[1, 1, 1, 1],
    inline=True,
    # output="../output/polblogs_5.png",
)

This visualization reveals the hierarchical structure more clearly. It's a bit tough to see right now, but the blue square node right in the middle of the network represents the entire graph merged into one group at the highest level of the block hierarchy. As you move outward from the center, the graph splits into smaller and smaller blocks, which correspond to different political blogs at the lowest level of the block hierarchy.

You may notice that the nested SBM reveals a more complex structure than a simple left-right division. The hierarchy shows internal differentiation within each political cluster, revealing sub-communities that were not as apparent in the force-directed layout. However, we can see a clear split into two groups at the highest-level below the full graph that does align with the binary classifications.

### Emphasize the Hierachichal Block Structure

As a final step, we can tweak the appearance of the block-level graph to make it easier to see and more visually appealing. We will modify the properties of the hierarchical block nodes and edges using `hvprops` (hierarchical vertex properties) and `heprops` (hierarchical edge properties).


In [ ]:
# customize hierarchical vertex properties (block nodes)
hvprops = {
    "fill_color": "white",  # make block nodes white (will top edges with color)
    "size": 30,             # increase the size of block nodes
}

# customize hierarchical edge properties (block edges)
heprops = {
    "color": "white",       # make block edges white (will also top edges with color)
    "pen_width": 2,         # thicken the edges
}

# draw the blockstate with the custom hierarchical properties
blockstate.draw(
    vertex_fill_color=vertex_political_colors,
    hvprops=hvprops,
    heprops=heprops,
    output_size=(1200, 1200),
    bg_color=[1, 1, 1, 1],
    inline=True,
    # output="../output/polblogs_5.png",
)

I decieded to go with with white for the block nodes and edges because

The adjustments make the block nodes and edges more prominent, improving the overall readability and aesthetics of the hierarchical block visualization.

## Next Steps

With that, we've successfully recreated the series of political blog network figures from the lecture using `graph-tool`. We learned how to run `graph-tool` code in a conda environment, extracted the giant component, fit our first nested Stochastic Blockmodel (NSBM), and created a series of visualizions of the network and it's hierarchical block structure. We also learned how to adjust force_directed layouts to add additional attractors for group memberships based on simple model criticism, and how to modify and refine the visual properties of networks at different levels of the block hierarchy.

In the next part of the tutorial, we'll explore the Enron email networks, applying similar techniques and deepening our understanding of community detection in large networks. We'll fit different kinds of models to different kinds of network representations and then rank the models based on their description lengths. As a bonus, we will compare these to results from modularity-maximization approaches (which you should not use).